# TUBERÍAS EN SERIE

El presente código muestra una forma para hacer comprobaciones de diseño de tuberías en serie con la ecaución general de pérdidas de Darcy-Weisbach. Se presenta una aplicación que podría darse para el caso de n tubos puestos en serie con caudales laterales desde la unión del primer tubo con el segundo. Pueden usarse diferentes materiales, diámetros y rugosidades y todo debería dar algo coherente. 

In [7]:
import numpy as np 
import pandas as pd 

## Ecuación para factor de fricción
Vamos a asumir que el flujo siempre es turbulento (por las condiciones en las que, generalmente, se dan los problemas de hidráulica de tuberías en ingeniería civil). Para poder calcular el factor de fricción $f$ de cada una de las tuberías de una serie, construimos arreglos numpy para los diámetros, rugosidades absolutas y números de Reynolds. Cuando estos valores entran a la función `Col_Whi`, esta devolverá un arreglo de valores de $f$ para cada una de las tunerías.

La ecuación de Colebrook-White que se usa para el cálculo del factor $f$ es:

$$\frac{1}{\sqrt{f}}=-2log_{10}\Bigg(\frac{k_s/D}{3.7} + \frac{2.51}{\mathbf{Re}\sqrt{f}}\Bigg)$$

En aras de la simplicidad del código, se optó por usar el algoritmo de iteraciones de punto fijo para encontrar las soluciones. También, se calcula la norma de la diferencia entre los vectores de $f$ previo y $f$ siguiente para establecer cuándo se puede cerrar el ciclo. 

In [8]:
def Col_Whi(D, ks, Re):
    '''Función para calcular los factores de fricción de una serie de tubos 
    usando la ecuación de Colebrook-White.
    
    Args:
        D (np.array - float): Diámetros de las tuberías
        ks (np.array - float): Rugosidades absolutas (mismas unidades que D)
        Re (np.array - float): Números de Reynolds de las tuberías
        
    Returns:
        np.array - float: Los factores de fricción de las tuberías'''
    
    # Definiendo tolerancia del solver y 
    tol = 1e-10
    err = 1e10

    # Convertir entradas a arrays (mínimo 1D) (así no hay problemas cuando la 
    # entrada es un solo número)
    D = np.atleast_1d(D)
    ks = np.atleast_1d(ks)
    Re = np.atleast_1d(Re)

    # Vector semilla de valores de f
    f0 = np.ones(len(D)) * 0.02

    # Bucle que se repite para encontrar los valores de f
    while err > tol:
    
        # Calculando los nuevos valores de f
        f1 = (-2 * np.log10(ks / D / 3.7 + 2.51 / Re / np.sqrt(f0))) ** -2

        # Estimando el valor del error como la norma del vector de diferencias
        # relativas de los factores f
        err = np.linalg.norm((f1 - f0) / f0)

        # Reemplazo los valores
        f0 = f1

    return f1 

In [9]:
# Probando la función propuesta
Diam = np.array([100, 250, 300])
ks1 = np.array([0.0015, 0.15, 0.2])
Re1 = np.array([150000, 45000, 25000])

Col_Whi(Diam, ks1, Re1)

array([0.01665904, 0.02327257, 0.02608331])

## Desarrollo de sistema en serie
Un sistema en serie de n tubos que conectan dos tanques tiene unan pérdida total igual a: 

$$h_T = h_1 + h_2 + ... + h_n$$

Donde $h_T$ es la pérdida total (diferencia de alturas entre los tanques), y $h_i, \ (i=1, 2, ..., n)$ las pérdidas en cada una de las secciones de tubería que constituyen al sistema total. Si se tiene en cuenta que la pérdida en cada una de las secciones está dada por: 

$$h_i=\Big(f_i \frac{L_i}{D_i} + \sum K_i\Big) \frac{v_i^2}{2g}=0.0826 \frac{Q_i^2}{D_i^4}\Big(f_i \frac{L_i}{D_i} + \sum K_i\Big)$$

Donde $Q_i$ es el caudal que recorre la i-ésima tubería, $D_i$, $L_i$ y $f_i$ son el diámetro, la longitud y el factor de fricción de la i-ésima tubería y $\sum K_i$ es la sumatoria de los coeficientes de pérdidas menores en la tubería i. Se debe notar que el 0.0826 se usa cuando las unidades que se trabajan son del sistema internacional (SI). Ahora, para añadir los caudales laterales de extracción, se llega a que el caudal que transcurre por la i-ésima tubería es igual a: 

$$Q_i = Q_T - \sum_{j=1}^{i-1}q_j$$

Donde $Q_T$ es el caudal total que fluye por el sistema antes de que haya derivaciones, $\sum q_j$ es la suma de los caudales que se han extraído antes de llegar a la i-ésima tubería y, por ende, $Q_i$ es el caudal que se extrae por la tubería i (que es una de las incógnitas que se busca resolver con este problema). Ahora bien, si se reemplaza la definición del caudal en cada uno de los tubos en la fórmula de pérdidas totales, se obtiene:

$$h_T=0.0826\Bigg[ Q_T^2\sum_{i=1}^n \alpha_i-2Q_T\sum_{i=1}^n\Big(\alpha_i \sum_{j=1}^{i-1}q_j\Big)+\sum_{i=1}^n \Bigg(\alpha_i \Big( \sum_{j=1}^{i-1}q_j\Big)^2\Bigg)\Bigg]$$

Donde:

$$\alpha_i = \Big(f_i \frac{L_i}{D_i^5}+\frac{\sum K_i}{D_i^4}\Big)$$

Esta expresión resulta ser una cuadrática cuya incógnita es el caudal total del sistema. Entonces, el proceso de solución parte de la suposición de $n$ factores de fricción $f$. Con estos valores se resuelve la cuadrática para encontrar el caudal total $Q_T$ (y por ende los caudales parciales de cada uno de los tramos). En seguida, se calcula el número de Reynolds en cada uno de los tramos y con estos valores se recalcula el valor de los factores de fricción $f$. En este paso, se calcula la diferencia entre el vector de factores de fricción calculado $f_{i+1}$ y el vector de factores de fricción supuesto $f_i$. Si la norma de esta diferencia es menor que la tolerancia, se ha llegado a la solución; de lo contrario, el factor de fricción calculado $f_{i+1}$ pasa a ser el factor de fricción inicial y se repite el proceso hasta llegar a laconvergencia deseada. 

In [10]:
# VALORES DE ENTRADA DEL PROBLEMA (ejemplo libro Saldarriaga, ed3, Cap5)
# Se le pone un punto a algunas unidades para que Python las interprete como 
# números con decimales. 

# Pérdida total en m
hT =  28.5

# Longitudes de tuberías en m
L = np.array([423, 174, 373., 121])

# Diámetros de tuberías en mm
D = np.array([600, 500, 300., 250])

# Rugosidades en mm
ks = np.array([0.3, 0.0015, 0.0015, 0.0015])

# Suma de coeficientes de pérdidas menores
Sum_K = np.array([4.2, 3.4, 5.3, 7.5])

# Caudales de derivación en L/s
ql = np.array([60, 74, 60.])

# Viscosidad cinemática de fluido m2/s
nu = 1.14e-6

# Transformación a unidades consistentes (de mm a m y de L/s a m3/s)
D /= 1000.
ks /= 1000.
ql /= 1000.

# Parámetros numéricos para detener el solver
err = 1e10
tol = 1e-8
K1 = 8 / np.pi ** 2 / 9.81   # Constante de carga velocidad

# Paso 1 del problema, se suponen los f de las tuberías (por simplicidad, se 
# usará 0.02 para todas las tuberías. Luego, el algoritmo irá corrigiendo)
f0 = np.ones(len(D)) * 0.015

Para ensamblar la ecuación cuadrática se parte de la expresión que se simplificó, se resta la pérdida total $h_T$ a ambos lados y se divide todo por el factor $0.0826$. Luego, los términos que son sólo números se agrupan para el término independiente y se resuelve la ecuación cuadrática propuesta. 

$$Q_T^2 \sum_{i=1}^n \alpha_i-Q_T2\sum_{i=1}^n\Big(\alpha_i \sum_{j=1}^{i-1} q_j\Big)+\sum_{i=1}^n \alpha_i \Big(\sum_{j=1}^{i-1} q_j\Big)^2 - \frac{h_T}{0.0826} = 0$$

En otras palabras: 

$$AQ_T^2+BQ_T+C=0$$

In [11]:
# La suma de los ql es solamente una suma acumulada de los caudales que van 
# saliendo del sistema
sql = np.cumsum(ql)
sql = np.append(0, sql)

it = 1

# Iterando en ciclo while para obtener el resultado
while err > tol:

    # Sacando los valores del vector alpha
    alpha = (1 / D ** 4) * (f0 * L / D + Sum_K)

    # Valores para sumatorias compuestas
    alpha_q = alpha * sql
    alpha_q2 = alpha * sql ** 2

    # Valores de la cuadrática
    A = sum(alpha)
    B = -2 * sum(alpha_q)
    C = sum(alpha_q2) - hT / K1

    # Solución de la cuadrática
    QT = np.roots([A, B, C])

    # Me quedo con la positiva
    QT = QT[QT >= 0]

    # Calculo los caudales de cada tubo
    Qi = QT - sql

    # Calculo los Reynolds para cada tubo
    Re = 4 * Qi / (np.pi * D * nu)

    # Con los Reynolds estimo los nuevos valores de f
    f1 = Col_Whi(D, ks, Re)
    
    # Calculo el error como la norma de la diferencia de f1 y f0
    err = np.linalg.norm((f1 - f0) / f0)

    # Reemplazo valores y sigo
    f0 = f1

    # Imprimo los resultados intemedios para verificar y revisar la convergencia
    # Incluye la impresión del error relativo en el vector de las f calculadas
    # vs las f supuestas
    print('\n\nIteración: %.i \tError: %.2e'%(it, err))
    
    print(pd.DataFrame({
        'Q (L/s)': Qi * 1000,
        'V (m/s)': Qi * 4 / np.pi / D ** 2,
        'Re': Re,
        'f': f1,
        'hf (m)': f1 * L / D ** 5 * K1 * Qi ** 2,
        'hk (m)': Sum_K * 0.0826 * Qi ** 2 / D ** 4
    }))
    it += 1




Iteración: 1 	Error: 3.67e-01
      Q (L/s)   V (m/s)             Re         f    hf (m)    hk (m)
0  381.537324  1.349412  710216.972210  0.017398  1.138377  0.389671
1  321.537324  1.637576  718235.151416  0.012389  0.589298  0.464560
2  247.537324  3.501937  921562.309932  0.011918  9.262028  3.311710
3  187.537324  3.820479  837824.341448  0.012128  4.366838  5.577720


Iteración: 2 	Error: 1.75e-02
      Q (L/s)   V (m/s)             Re         f     hf (m)    hk (m)
0  396.320695  1.401698  737735.645653  0.017374   1.226593  0.420453
1  336.320695  1.712867  751257.559548  0.012294   0.639770  0.508260
2  262.320695  3.711079  976599.656819  0.011805  10.302584  3.719084
3  202.320695  4.121643  903869.157712  0.011977   5.019276  6.491752


Iteración: 3 	Error: 8.17e-04
     Q (L/s)   V (m/s)             Re         f     hf (m)    hk (m)
0  397.03444  1.404222  739064.253198  0.017373   1.230936  0.421969
1  337.03444  1.716502  752851.888603  0.012290   0.642255  0.510420
2 

In [12]:
# En este dataframe se presentan los resultados de forma ordenada para poder
# hacer las revisiones contra el texto
resultados = pd.DataFrame({
              'Diam. (mm)': D * 1000,
              'Long (m)': L,
              'Rug. (mm)': ks * 1000,
              'Suma acc.': Sum_K,
              'Caudal (L/s)': Qi * 1000,
              'Vel. (m/s)': 4 * Qi / (np.pi * D ** 2), 
              'he (m)': (f1 * L / D + Sum_K) * K1 * Qi ** 2 / D ** 4,
              'hf (m)': f1 * L / D ** 5 * K1 * Qi ** 2,
              'hk (m)': Sum_K * K1 * Qi ** 2 / D ** 4,
              'f': f1, 
              'Re': Re})

resultados.style.format({
              'Diam. (mm)': '{:.0f}',
              'Long (m)': '{:.0f}',
              'Rug. (mm)': '{:.4f}',
              'Suma acc.': '{:.1f}',
              'Caudal (L/s)': '{:.2f}',
              'Vel. (m/s)': '{:.3f}',
              'he (m)': '{:.2f}', 
              'hf (m)': '{:.2f}',
              'hk (m)': '{:.2f}',
              'f': '{:.5f}', 
              'Re': '{:.1f}'
})

,Diam. (mm),Long (m),Rug. (mm),Suma acc.,Caudal (L/s),Vel. (m/s),he (m),hf (m),hk (m),f,Re
0,600,423,0.3000,4.2,397.07,1.404,1.65,1.23,0.42,0.01737,739129.0
1,500,174,0.0015,3.4,337.07,1.717,1.15,0.64,0.51,0.01229,752929.6
2,300,373,0.0015,5.3,263.07,3.722,14.10,10.36,3.74,0.01180,979386.3
3,250,121,0.0015,7.5,203.07,4.137,11.60,5.05,6.54,0.01197,907213.2
